In [ ]:

data Op = Add | Sub | Mul | Div | Pow -- define the data type Op
instance Show Op where -- define the show function for Op
    show Add = "+" -- define the show function for Add
    show Sub = "-" -- define the show function for Sub
    show Mul = "*" -- define the show function for Mul
    show Div = "/" -- define the show function for Div
    show Pow = "^" -- define the show function for Pow

apply :: Op -> Int -> Int -> Int -- define the apply function for Op
apply Add x y = x + y -- add adds x and y
apply Sub x y = x - y -- sub subtracts y from x
apply Mul x y = x * y -- mul multiplies x and y
apply Div x y = x `div` y -- div divides x by y

valid :: Op -> Int -> Int -> Bool -- define the valid function for Op
valid Add _ _ = True -- add is always valid
valid Sub x y = x > y -- sub is valid if x is greater than y
valid Mul _ _ = True -- mul is always valid
valid Div x y = y /= 0 && x `mod` y == 0 -- div is valid if y is not 0 and x mod y is 0

data Expr = Val Int | App Op Expr Expr  -- define the data type Expr
instance Show Expr where -- define the show function for Expr
    show (Val n) = show n -- show Val n as n
    show (App o l r) = brak l ++ show o ++ brak r -- show App o l r as brak l + show o + brak r
                        where
                            brak (Val n) = show n
                            brak e = "(" ++ show e ++ ")" -- show brak (Val n) as show n and brak e as "(" ++ show e ++ ")"

-- the subs function returns all subsequences of a list
subs :: [a] -> [[a]] -- define the subs function for [a]
subs [] = [[]] -- subs [] is [[]]
subs (x:xs) = yss ++ map (x:) yss
                  where yss = subs xs

-- the interleave function returns all ways of inserting a new element into a list
interleave :: a -> [a] -> [[a]]
interleave x [] = [[x]]
interleave x (y:ys) = (x:y:ys) : map (y:) (interleave x ys)

-- the perms function returns all permutations of a list
perms :: [a] -> [[a]]
perms [] = [[]]
perms (x:xs) = concat (map (interleave x) (perms xs))

-- the choices function returns all possible ways of selecting zero or more elements in any order
choices :: [a] -> [[a]]
choices xs = [zs | ys <- subs xs, zs <- perms ys]

-- the split function returns all possible ways of splitting a list into two non-empty lists that append to give the original list
split :: [a] -> [([a],[a])]
split [] = []
split [_] = []
split (x:xs) = ([x],xs) : [(x:ls,rs)
                                | (ls,rs) <- split xs]

-- the combine function returns all possible expressions whose list of values is precisely a given list
combine :: Expr -> Expr -> [Expr]
combine l r =
        [App o l r | o <- [Add,Sub,Mul,Div]]

-- the exprs function returns all possible expressions over a given list of numbers
exprs :: [Int] -> [Expr]
exprs [] = []
exprs [n] = [Val n]
exprs ns = [e | (ls,rs) <- split ns
        , l <- exprs ls
        , r <- exprs rs
        , e <- combine l r]

-- the eval function returns the list of values of an expression
eval :: Expr -> [Int]
eval (Val n) = [n | n > 0]
eval (App o l r) = [apply o x y | x <- eval l
                                , y <- eval r
                                , valid o x y]


-- brute force solution:
-- the solutions function returns all possible expressions over a given list of numbers that solve an instance of the countdown problem
solutions :: [Int] -> Int -> [Expr]
solutions ns n = [e | ns' <- choices ns
                            , e <- exprs ns'
                            , eval e == [n]]

-- improvement stage 1:

type Result = (Expr,Int) -- define the data type Result

-- the results function returns all possible results of splitting a list into two non-empty lists that append to give the original list
results :: [Int] -> [Result]
results ns = [(e,n) | ns' <- choices ns
                    , (e,n) <- results' ns']

-- the results' function returns all possible results of splitting a list into two non-empty lists that append to give the original list
results' :: [Int] -> [Result]
results' [] = []
results' [n] = [(Val n,n) | n > 0]
results' ns = [res | (ls,rs) <- split ns
                    , lx <- results' ls
                    , ry <- results' rs
                    , res <- combine' lx ry]

-- the combine' function returns all possible expressions whose list of values is precisely a given list
combine' :: Result -> Result -> [Result]
combine' (l,x) (r,y) =
        [(App o l r, apply o x y) | o <- [Add,Sub,Mul,Div]
                                    , valid o x y]

-- the solutions' function returns all possible expressions over a given list of numbers that solve an instance of the countdown problem
solutions' :: [Int] -> Int -> [Expr]
solutions' ns n = [e | ns' <- choices ns
                            , (e,m) <- results' ns'
                            , m == n]

-- improvement stage 2:

-- the valid' function returns whether an operation is valid
valid' :: Op -> Int -> Int -> Bool
valid' Add x y = x <= y
valid' Sub x y = x > y
valid' Mul x y = x /= 1 && y /= 1 && x <= y
valid' Div x y = y /= 1 && x `mod` y == 0

-- the results'' function returns all possible results of splitting a list into two non-empty lists that append to give the original list
results'' :: [Int] -> [Result]
results'' [] = []
results'' [n] = [(Val n,n) | n > 0]
results'' ns = [res | (ls,rs) <- split ns
                    , lx <- results'' ls
                    , ry <- results'' rs
                    , res <- combine'' lx ry]

-- the combine'' function returns all possible expressions whose list of values is precisely a given list
combine'' :: Result -> Result -> [Result]
combine'' (l,x) (r,y) =
        [(App o l r, apply o x y) | o <- [Add,Sub,Mul,Div]
                                    , valid' o x y]

-- the solutions'' function returns all possible expressions over a given list of numbers that solve an instance of the countdown problem
solutions'' :: [Int] -> Int -> [Expr]
solutions'' ns n = [e | ns' <- choices ns
                            , (e,m) <- results'' ns'
                            , m == n]

main :: IO ()
main = do
        print "1) Brute Force Solution"
        print (length(solutions [1,3,7,10,25,50] 765))
        print (length(solutions [1,3,7,10,25,50] 831))
        print "2) Improved (Stage 1)"
        print (length(solutions' [1,3,7,10,25,50] 765))
        print (length(solutions' [1,3,7,10,25,50] 831))
        print "3) Improved (Stage 2)"
        print (length(solutions'' [1,3,7,10,25,50] 765))
        print (length(solutions'' [1,3,7,10,25,50] 831))



Line 41: Use concatMap
Found:
concat (map (interleave x) (perms xs))
Why not:
concatMap (interleave x) (perms xs)

In [2]:
do
        print "1) Brute Force Solution"
        -- print length((solutions [1,3,7,10,25,50] 765))
        
        print "2) Improved (Stage 1)"
        -- print length((solutions' [1,3,7,10,25,50] 765))
        print "3) Improved (Stage 2)"
        print length((solutions'' [1,3,7,10,25,50] 765))

Line 7: Redundant bracket
Found:
((solutions'' [1, 3, 7, 10, 25, 50] 765))
Why not:
(solutions'' [1, 3, 7, 10, 25, 50] 765)

: 

In [3]:
-- test the code against the following numbers 1,3,7,10,25,50 and target 765 using the following operators +,-,*,/


In [ ]:
-- test the code against the following numbers 1,3,7,10,25,50 and target 831 using the following operators +,-,*,/

: 